<a href="https://colab.research.google.com/github/Mihir1703/Deep-Learning/blob/master/Fruit_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                             title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                       COVID-19 Dataset                                 5MB  2022-11-13 15:47:17          14268        409  1.0              
thedevastator/analyzing-credit-card-spending-habits-in-india    Credit Card Spending Habits in India           319KB  2022-12-14 07:30:37           1195         48  1.0              
michals22/coffee-dataset                                        Coffee dataset                                  24KB  2022-12-15 20:02:12           3351         76  1.0              
thedevastator/unlock-profits-with-e-commerce-sales-

In [2]:
!kaggle datasets download -d moltean/fruits

100% 1.28G/1.28G [01:03<00:00, 22.3MB/s]
100% 1.28G/1.28G [01:03<00:00, 21.7MB/s]


In [3]:
!mkdir datasets
!unzip -q fruits.zip -d datasets

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
import keras.utils as image
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [5]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [6]:
classes = os.listdir('datasets/fruits-360_dataset/fruits-360/Training')
classes[63],len(classes)

('Chestnut', 131)

In [7]:
train_batch=ImageDataGenerator().flow_from_directory(directory='datasets/fruits-360_dataset/fruits-360/Training',target_size=(229,229),classes=classes)

Found 67692 images belonging to 131 classes.


In [8]:
label_dict = dict((v,k) for k,v in train_batch.class_indices.items())
label_code = train_batch.class_indices
label_dict[45]

'Cherry Wax Black'

In [9]:
model = Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/classification/5",trainable=False),
    Dense(units=len(classes),activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001, decay=0.001 / (5 * 0.5)) , metrics=['accuracy'])
model.build([None, 299, 299, 3])

In [10]:
if os.path.isfile('fruit.h5') is False:
    with tf.device("/device:GPU:0"):
        model.fit(train_batch,epochs=5)
    model.save('fruit.h5')
    with open("num_to_label.json", "w+") as outfile:
        json.dump(label_dict, outfile)
    with open("label_to_num.json", "w+") as outfile:
        json.dump(label_code, outfile)

Epoch 1/5
2116/2116 [==============================] - 178s 78ms/step - loss: 22.5802 - accuracy: 0.6695
Epoch 2/5
2116/2116 [==============================] - 162s 77ms/step - loss: 4.9874 - accuracy: 0.8771
Epoch 3/5
2116/2116 [==============================] - 161s 76ms/step - loss: 2.6347 - accuracy: 0.9236
Epoch 4/5
2116/2116 [==============================] - 162s 76ms/step - loss: 1.6326 - accuracy: 0.9464
Epoch 5/5
2116/2116 [==============================] - 161s 76ms/step - loss: 1.0809 - accuracy: 0.9615


In [11]:
load_model = keras.models.load_model(
    'fruit.h5', custom_objects={'KerasLayer':hub.KerasLayer}, compile=True, options=None
)

In [14]:
with open("num_to_label.json") as outfile:
    label_dict = json.loads(outfile.read())
with open("label_to_num.json") as outfile:
    label_code = json.loads(outfile.read())
label_dict,label_code

({'0': 'Pineapple',
  '1': 'Tomato not Ripened',
  '2': 'Tomato Maroon',
  '3': 'Mulberry',
  '4': 'Apple Golden 2',
  '5': 'Pear',
  '6': 'Onion Red',
  '7': 'Nectarine Flat',
  '8': 'Huckleberry',
  '9': 'Apple Golden 1',
  '10': 'Pear Abate',
  '11': 'Plum 2',
  '12': 'Lychee',
  '13': 'Pear Forelle',
  '14': 'Avocado',
  '15': 'Mandarine',
  '16': 'Pepper Orange',
  '17': 'Hazelnut',
  '18': 'Tomato 4',
  '19': 'Cherry 2',
  '20': 'Passion Fruit',
  '21': 'Apple Red Delicious',
  '22': 'Nut Forest',
  '23': 'Potato White',
  '24': 'Blueberry',
  '25': 'Raspberry',
  '26': 'Dates',
  '27': 'Tangelo',
  '28': 'Fig',
  '29': 'Watermelon',
  '30': 'Cantaloupe 1',
  '31': 'Kiwi',
  '32': 'Potato Red Washed',
  '33': 'Apple Red 2',
  '34': 'Onion White',
  '35': 'Eggplant',
  '36': 'Cherry Rainier',
  '37': 'Papaya',
  '38': 'Apple Braeburn',
  '39': 'Ginger Root',
  '40': 'Mango',
  '41': 'Nut Pecan',
  '42': 'Pomelo Sweetie',
  '43': 'Pepper Red',
  '44': 'Pear Kaiser',
  '45': 'Cherry

In [18]:
import keras.utils as image

test_path = 'datasets/fruits-360_dataset/fruits-360/Test/'
orignal = []
predict = []

print(label_code,label_dict)

def process_img(file):
    img = image.load_img(file,target_size=(229,229))
    return np.array(img)


for i in os.listdir(test_path):
    batch = []
    for j in os.listdir(test_path+i):
        path = test_path+i+'/'+j
        img = process_img(path)
        batch.append(img)
        orignal.append(label_code[i])
    with tf.device("/device:GPU:0"):
        pred = np.argmax(model.predict(np.array(batch)),axis=1)
    predict = predict + pred.tolist()

{'Pineapple': 0, 'Tomato not Ripened': 1, 'Tomato Maroon': 2, 'Mulberry': 3, 'Apple Golden 2': 4, 'Pear': 5, 'Onion Red': 6, 'Nectarine Flat': 7, 'Huckleberry': 8, 'Apple Golden 1': 9, 'Pear Abate': 10, 'Plum 2': 11, 'Lychee': 12, 'Pear Forelle': 13, 'Avocado': 14, 'Mandarine': 15, 'Pepper Orange': 16, 'Hazelnut': 17, 'Tomato 4': 18, 'Cherry 2': 19, 'Passion Fruit': 20, 'Apple Red Delicious': 21, 'Nut Forest': 22, 'Potato White': 23, 'Blueberry': 24, 'Raspberry': 25, 'Dates': 26, 'Tangelo': 27, 'Fig': 28, 'Watermelon': 29, 'Cantaloupe 1': 30, 'Kiwi': 31, 'Potato Red Washed': 32, 'Apple Red 2': 33, 'Onion White': 34, 'Eggplant': 35, 'Cherry Rainier': 36, 'Papaya': 37, 'Apple Braeburn': 38, 'Ginger Root': 39, 'Mango': 40, 'Nut Pecan': 41, 'Pomelo Sweetie': 42, 'Pepper Red': 43, 'Pear Kaiser': 44, 'Cherry Wax Black': 45, 'Walnut': 46, 'Pear Williams': 47, 'Rambutan': 48, 'Cherry Wax Yellow': 49, 'Lemon': 50, 'Cherry Wax Red': 51, 'Peach Flat': 52, 'Physalis with Husk': 53, 'Mangostan': 54

In [16]:
from sklearn.metrics import accuracy_score

print("Accuracy : ",accuracy_score(y_pred=predict,y_true=orignal))

Accuracy :  0.91
